In [1]:
import os
import cv2
import torch
import pickle
import numpy as np

from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard.writer import SummaryWriter
from torch.nn import Conv2d, BatchNorm2d, ReLU, Sequential, MaxPool2d, Linear, Flatten, functional, CrossEntropyLoss

## 解压图片文件

In [2]:
MODE = 'test'
SAVE_PATH = f'./data/cifar-10-python/cifar-10-batches-py/{MODE}/'
LABEL_NAME = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

label_dict = {}
for i, name in enumerate(LABEL_NAME):
    label_dict[name] = i

label_dict

{'airplane': 0,
 'automobile': 1,
 'bird': 2,
 'cat': 3,
 'deer': 4,
 'dog': 5,
 'frog': 6,
 'horse': 7,
 'ship': 8,
 'truck': 9}

In [3]:
def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

In [4]:
import glob

data_list = glob.glob(f'./data/cifar-10-python/cifar-10-batches-py/{MODE}_batch*')
data_list

['./data/cifar-10-python/cifar-10-batches-py\\test_batch']

In [5]:
# 循环读取文件，保存为图片格式
for path in data_list:
    data_dict = unpickle(path)
    # print(data_dict.keys())
    # print(data_dict[b'data'])
    # batch_label：批次号；labels：对应标签名称；data：图像数据（3×32×32）；filenames：文件名称

    for img_id, img_data in enumerate(data_dict[b'data']):
        img_name = data_dict[b'filenames'][img_id]
        img_label = LABEL_NAME[int(data_dict[b'labels'][img_id])]
        # print(img_name, img_label)

        img_data = np.reshape(img_data, [3, 32, -1])  # 图片数据格式为3×32×32
        img_data = np.transpose(img_data, (1, 2, 0))  # 将图片格式转换为32×32×3，为什么要这样？我也不理解，但是问题不大，可能是因为cv2的格式、要求
        # cv2.imshow('img_data', cv2.resize(img_data, (200, 200)))
        # cv2.waitKey(0)

        # 对训练集图片按照不同标签对应不同的文件夹分类
        cur_path = SAVE_PATH + img_label
        if not os.path.exists(cur_path):
            os.makedirs(cur_path)

        # Saving image
        cv2.imwrite(cur_path + '/' + img_name.decode('utf-8'), img_data)

## 定义数据集类并加载数据集

In [6]:
# 训练数据集增强
train_transformer = transforms.Compose([
    transforms.RandomCrop(28),  # 改变大小
    transforms.RandomHorizontalFlip(),  # 水平翻转
    transforms.ToTensor()
])

test_transformer = transforms.Compose([
    transforms.Resize((28, 28)),  # 改变大小
    transforms.ToTensor()
])


# 定义读取数据集类
class MyDataset(Dataset):
    def __init__(self, img_list, transform=None, loader=None):
        super(MyDataset, self).__init__()

        self.transform = transform

        self.loader = loader
        if not self.loader:
            self.loader = self.image_loader

        self.img_info = []
        for path in img_list:
            img_label = path.split('\\')[-2]
            self.img_info.append((path, LABEL_NAME.index(img_label)))
            # img_info.append((path, label_dict[img_label]))

    def __getitem__(self, index):
        img_path, img_label = self.img_info[index]

        img_data = self.loader(img_path)
        # 是否进行图像增强处理
        if self.transform:
            img_data = self.transform(img_data)
        return img_data, img_label

    def __len__(self):
        return len(self.img_info)

    @staticmethod
    def image_loader(path):
        return Image.open(path).convert('RGB')

In [7]:
# 获取全部的训练集图片路径以及测试集图片路径
img_train_list = glob.glob('./data/cifar-10-python/cifar-10-batches-py/train/*/*.png')
img_test_list = glob.glob('./data/cifar-10-python/cifar-10-batches-py/test/*/*.png')

train_dataset = MyDataset(img_train_list, transform=train_transformer)  # 测试集图片需要进行图片增强处理
test_dataset = MyDataset(img_test_list, transform=transforms.ToTensor())  # 训练集图片不需要
len(train_dataset), len(test_dataset)

(50000, 10000)

## 加载数据集类

In [8]:
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=6,
    shuffle=True,
    num_workers=2
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=6,
    num_workers=2
)

len(train_loader), len(test_loader)

(8334, 1667)

In [9]:
len(train_loader.dataset), len(test_loader.dataset)

(50000, 10000)

## 搭建VGGNet

In [10]:
class VGGNet(torch.nn.Module):
    def __init__(self):
        super(VGGNet, self).__init__()

        self.model = Sequential(
            # 00
            Conv2d(3, 64, kernel_size=3, padding=1),
            BatchNorm2d(64),  # num_features：一般输入参数为batch_size*num_features*height*width，即为其中特征的数量
            ReLU(),
            MaxPool2d(kernel_size=2, stride=2),
            # 01
            Conv2d(64, 128, kernel_size=3, padding=1),
            BatchNorm2d(128),
            ReLU(),
            Conv2d(128, 128, kernel_size=3, padding=1),
            BatchNorm2d(128),
            ReLU(),
            MaxPool2d(kernel_size=2, stride=2),
            # 02
            Conv2d(128, 256, kernel_size=3, padding=1),
            BatchNorm2d(256),
            ReLU(),
            Conv2d(256, 256, kernel_size=3, padding=1),
            BatchNorm2d(256),
            ReLU(),
            MaxPool2d(kernel_size=2, stride=2, padding=1),
            # 03
            Conv2d(256, 512, kernel_size=3, padding=1),
            BatchNorm2d(512),
            ReLU(),
            Conv2d(512, 512, kernel_size=3, padding=1),
            BatchNorm2d(512),
            ReLU(),
            MaxPool2d(kernel_size=2, stride=2),

            Flatten(),
            Linear(512 * 4, 10)
        )

        # Input: 3×28×28
        self.conv_00 = Sequential(
            Conv2d(3, 64, kernel_size=3, padding=1),
            BatchNorm2d(64),  # num_features：一般输入参数为batch_size*num_features*height*width，即为其中特征的数量
            ReLU()
        )
        self.max_pool_00 = MaxPool2d(kernel_size=2, stride=2)

        # Input: 14×14
        self.conv_01_00 = Sequential(
            Conv2d(64, 128, kernel_size=3, padding=1),
            BatchNorm2d(128),
            ReLU()
        )
        self.conv_01_01 = Sequential(
            Conv2d(128, 128, kernel_size=3, padding=1),
            BatchNorm2d(128),
            ReLU()
        )
        self.max_pool_01 = MaxPool2d(kernel_size=2, stride=2)

        # Input: 7×7
        self.conv_02_00 = Sequential(
            Conv2d(128, 256, kernel_size=3, padding=1),
            BatchNorm2d(256),
            ReLU()
        )
        self.conv_02_01 = Sequential(
            Conv2d(256, 256, kernel_size=3, padding=1),
            BatchNorm2d(256),
            ReLU()
        )
        self.max_pool_02 = MaxPool2d(kernel_size=2, stride=2, padding=1)

        # Input: 4×4
        self.conv_03_00 = Sequential(
            Conv2d(256, 512, kernel_size=3, padding=1),
            BatchNorm2d(512),
            ReLU()
        )
        self.conv_03_01 = Sequential(
            Conv2d(512, 512, kernel_size=3, padding=1),
            BatchNorm2d(512),
            ReLU()
        )
        self.max_pool_03 = MaxPool2d(kernel_size=2, stride=2, padding=1)

        self.Flatten = Flatten()
        self.linear = Linear(512 * 9, 10)

    def forward(self, X):
        # batchsize = X.size(0)

        # _ = self.conv_00(X)
        # _ = self.max_pool_00(_)
        #
        # _ = self.conv_01_00(_)
        # _ = self.conv_01_01(_)
        # _ = self.max_pool_01(_)
        #
        # _ = self.conv_02_00(_)
        # _ = self.conv_02_01(_)
        # _ = self.max_pool_02(_)
        #
        # _ = self.conv_03_00(_)
        # _ = self.conv_03_01(_)
        # _ = self.max_pool_03(_)
        # print(_.shape)
        # _ = self.Flatten(_)
        # print(_.shape)
        # _ = self.linear(_)
        outputs = self.model(X)
        # outputs = _
        # outputs = functional.log_softmax(outputs)
        return outputs

In [11]:
model = VGGNet()

# 验证模型流程是否正确
demo_input = torch.ones(6, 3, 28, 28)  # batch_size=6, in_channels=3, height=28, weight=28.
model(demo_input).shape

torch.Size([6, 10])

In [19]:
from rich.console import Console
from rich.table import Column, Table

console = Console()

table = Table(show_header=True, header_style="bold magenta")
table.add_column("Date", style="dim", justify='center')
table.add_column("Title", justify='center')
table.add_column("Production Budget", justify='center')
table.add_column("Box Office", justify='center')
table.add_row(
    "Dec 20, 2019", "Star Wars: The Rise of Skywalker", "$275,000,000", "$375,126,118"
)
table.add_row(
    "May 25, 2018",
    "[red]Solo[/red]: A Star Wars Story",
    "$275,000,000",
    "$393,151,347",
)
table.add_row(
    "Dec 15, 2017",
    "Star Wars Ep. VIII: The Last Jedi",
    "$262,000,000",
    "[bold]$1,332,539,889[/bold]",
)

console.print(table)

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃     Date     ┃               Title               ┃ Production Budget ┃   Box Office   ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ Dec 20, 2019 │ Star Wars: The Rise of Skywalker  │   $275,000,000    │  $375,126,118  │
│ May 25, 2018 │      Solo: A Star Wars Story      │   $275,000,000    │  $393,151,347  │
│ Dec 15, 2017 │ Star Wars Ep. VIII: The Last Jedi │   $262,000,000    │ $1,332,539,889 │
└──────────────┴───────────────────────────────────┴───────────────────┴────────────────┘

In [26]:
print('+' * 25, "Ursula".center(10, '='), '+' * 25)  # 居中对齐

+++++++++++++++++++++++++ ==Ursula== +++++++++++++++++++++++++


In [ ]:
for batch_num, (X, y) in enumerate(train_loader):
    # X, y = X.to(device), y.to(device)
    print(y)
    # y_pred = model(X)
    # loss = loss_fun(y_pred, y)
    #
    # # 反向传播
    # optimizer.zero_grad()
    # loss.backward()
    # optimizer.step()
    #
    # print(f'Epoch is {epoch_num}, batch is {batch_num}, loss is: {loss.item()}.')
    break

In [13]:
# Finding device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Loss function
loss_fun = CrossEntropyLoss()
loss_fun.to(device)

# Optimizer
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.SGD(
#     model.parameters(),
#     lr=learning_rate,
#     momentum=.9,
#     weight_decay=5e-4
# )
# 学习率的衰减, 每经过5轮更新梯度，学习率衰减为原来的0.9
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=.9)

# Using tensorboard to visualize network structure
writer = SummaryWriter('./logs')

In [ ]:
epoch = 10
for epoch_num in range(epoch):

    # model.train()
    for batch_num, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        print(y)
        # y_pred = model(X)
        # loss = loss_fun(y_pred, y)
        #
        # # 反向传播
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        #
        # print(f'Epoch is {epoch_num}, batch is {batch_num}, loss is: {loss.item()}.')
        break
    break

++++
